In [4]:
%load_ext autoreload
%autoreload 2

import text_processing
import utils
import tf_idf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'icu'

# Oscar Wilde's Use of Literary Devices Over Time
##### Jessica Brown and Lauren Nalajala
## Background
Talk about Oscar Wilde's life here (add image for timeline of his life?)

And the goal of this project here

## Intial Text Processing
Talk about getting the data, processing it (ie. what we removed)

In [ ]:
#Import data from wilde, as well as other authors

## Word Usage
Introduce plots of word usage

In [ ]:
# create plots of his most commonly used words in one book, and focus in on specific words here?

Analyze most commonly used words here

## Numbers - some sort of catchy title here?
Numerical analysis of Wilde's writing - average length of words, average length of sentences over time
Maybe a comparison with another book?

In [ ]:
#plots for word length, sentence length of wilde over time\
#also scatter plot of average word/sentence length of wilde's books and compared to other authors

Discuss how Wilde's writing style changed over time, as well as how he compares to other authors of his time

## Polarity
Talk about how we analyzed polarity and make plots of character polarity - maybe connect them back to his life (ie. who/what they represent)

In [ ]:
#polarity plots here

## TF-IDF
Explain what TF-IDF is, the formula for it, and graph it for individual books\
Also plot TF-IDF of certain words over time 

In [ ]:
#TF-IDF plots here